## Arquivo e Biblioteca

In [15]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv('car_evaluation.csv')

In [17]:
df.head(5)

,buying price,maintenance cost,number of doors,number of persons,car_boot,safety,decision
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [18]:
df.describe()

,buying price,maintenance cost,number of doors,number of persons,car_boot,safety,decision
count,1727,1727,1727,1727,1727,1727,1727
unique,4,4,4,3,3,3,4
top,low,low,5more,4,med,med,unacc
freq,432,432,432,576,576,576,1209


## Explicação, Limpeza e Tratamento dos Dados

### Explicação do Dataset

O dataset escolhido revela aspectos para avaliação da qualidade dos carros inspecionados, como número de pessoas, preço de custo, segurança, custo de manutenção et cetera. Nesse projeto, nosso objetivo é elaborar um modelo de *Random Forest* para predizer se o carro tem qualidade (*accurate*) ou não tem qualidade (*unaccurate*)

### Limpeza e Tratamento

Como foi possível notar pelo df.describe(), não há valores faltantes, logo isso não será um problema para nossa modelagem. Além disso, todas as variáveis são qualitativas, apropriadas para a criação de algoritmos de classificação. Sendo assim, só precisaremos fazer dois ajustes:
1. A nossa variável-objetivo (*decision*) tem 4 valores possíveis: *unaccurate*,*accurate*,*good* e *very good*. Como só desejamos saber se o carro tem ou não qualidade, agruparemos *good* e *very good* junto a *accurate*.
2. Precisamos fazer o label encoding (transformar os valores categóricos em números), para que seja possível a modelagem do algoritmo de *Random Forest*

In [19]:
#Agrupando good e very good com accurate
df = df.replace(to_replace=["good","vgood"],
           value="acc")

In [22]:
#Transformar valores categóricos em números
from sklearn import preprocessing  
label_encoder = preprocessing.LabelEncoder()  
df['buying price'] = label_encoder.fit_transform(df['buying price'])
df['maintenance cost'] = label_encoder.fit_transform(df['maintenance cost'])
df['number of doors'] = label_encoder.fit_transform(df['number of doors'])
df['number of persons'] = label_encoder.fit_transform(df['number of persons'])
df['car_boot'] = label_encoder.fit_transform(df['car_boot'])
df['safety'] = label_encoder.fit_transform(df['safety'])
df['decision'] = label_encoder.fit_transform(df['decision'])

## Explicação do Algoritmo: Random Forest

Em um primeiro momento, é fundamental esclarecer que o *Random Forest* surge como base no algoritmo de Árvores de Decisão, visando a criação de um modelo mais robusto que o primeiro. Para tanto, o modelo tem como principal ferramenta a aleatoriedade de suas escolhas, como veremos no passo-a-passo:
1. Criar várias árvores de decisão, cada qual com o mesmo tamanho de amostra, mas as amostras são obtidas aleatoriamente, de modo que a amostra da primeira árvore pode ser distinta à amostra da segunda árvore - chamamos essa técnica de *bagging (bootstrapp aggregation)*
2. Fazendo uso novamente da aleatoriedade, sortear um número específico de features para cada árvore de decisão - a ideia aqui é randomizar a escolha das features para diminuir a correlação entre as árvores de decisão do modelo
3. Para cada árvore, realizar o procedimento padrão do algoritmo de árvores de decisão. Feito isso, contabilizar os votos respectivos a cada nó, e obter os resultados com base na média dos resultados individuais das árvores de decisão.